# Equilibrium Solver #
The Equilibrium solver is designed to take a reaction network as input and solve for a correct equilibrium solution. It does this by writing the system as a system of equations. Solving these equations can give the expected concentrations of all species at Equilibrium.

It is useful to have this capability because we want to compare the results of our simulations to equilibrium. This allows for the detection of traps and other interesting kinetic effects. 

In [1]:
# make sure jupyter path is correct for loading local moudules
import sys
# path to steric_simulator module relative to notebook
sys.path.append("../../")

In [2]:
from steric_free_simulator import ReactionNetwork, EqSolver, VectorizedRxnNetwork 

EnergyExplorer Module is not available. Check Rosetta installation. <ipykernel.iostream.OutStream object at 0x10e11eda0>


As usual, we can start by loading up the reaction network

In [3]:
base_input = '../input_files/trimer.pwr'
rn = ReactionNetwork(base_input, one_step=True)
rn.resolve_tree()

A minor annoyance is that the reaction network need the association constants to be resolved, which normally happens at simulation time. We can work around this by generated the vectorized network then writing it back to the normal reaction network.

In [ ]:
vec_rn = VectorizedRxnNetwork()